# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [ ]:
#for Windows
import findspark
findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

df = spark.sql('''select 'spark' as hello ''')
df.show()

In [ ]:
import os
import IPython
sound_path = 'C:\Windows\media\Windows Pop-up Blocked.wav'
file = os.path.expandvars(sound_path)
audio = IPython.display.Audio(filename=file, autoplay=True)
def ding():
    IPython.display.display(audio)
ding()



In [ ]:
# import libraries
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, Window
from pyspark.sql import functions as f
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.window import Window as w
from pyspark.ml.classification import LogisticRegression, LinearSVC, GBTClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import sys

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# create a Spark session
# docs here: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html
# configure = SparkConf().setAppName("Sparkify").setMaster("local")
    #     for SparkSession # .config(conf=configure) \
spark = SparkSession \
    .builder \
    .getOrCreate()
    
    

In [ ]:
ding

# Load and Clean Dataset
Clean your dataset, checking for invalid or missing data. For example, records without userids or sessionids. In this workspace, the filename is `mini_sparkify_event_data.json`.

In [ ]:
df = spark.read.json('mini_sparkify_event_data.json')
df.head()

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [ ]:
df.printSchema()

In [ ]:
#num rows
df.count()

In [ ]:
df = df.dropna(thresh=15)
df.count()

In [ ]:
278154/286500

In [ ]:
df.select('page').distinct().show()

In [ ]:
df.groupBy(df.page).count().show()

In [ ]:
print(f"Number who confirmed cancellation: {df.where(df.page=='Cancellation Confirmation') \
    .count()}")
df.where(df.page=='Cancellation Confirmation') \
    .groupBy('gender')\
    .count() \
    .show()

In [ ]:
df.select('userId').distinct().count()

In [ ]:
print(df.columns)

In [ ]:
df = df.withColumn('datetime',f.from_unixtime(df.ts/1000))

In [ ]:
df.head()

### Defining *Churn*
For now, we'll define *churn* as those who reach the "Cancellation Confirmation" Page.

In [ ]:
churns = list(df.where(df.page=='Cancellation Confirmation').select('userId').toPandas()['userId'])

In [ ]:
churns = set([int(x) for x in churns])

In [ ]:
print(churns)

In [ ]:
flag_churn = f.udf(lambda x: 1 if int(x) in churns else 0, IntegerType())

In [ ]:
df = df.withColumn('churns', flag_churn(df.userId))

In [ ]:
df.head()

## Examining page visits between users who 'churn' and users who don't
Particular page view stats might provide insight into users who 'churn'

In [ ]:
num_churn = df.where(df.churns==1).select('userId').distinct().count()

In [ ]:
num_stay = df.where(df.churns==0).select('userId').distinct().count()

In [ ]:
total = df.select('userId').distinct().count()

In [ ]:
#amounts in each group
num_churn, num_stay, total

In [ ]:
#raw data for users who churn and don't, toPandas for visualization purposes later
churns_comp_df = df.crosstab('churns','page').toPandas()
churns_comp_df

In [ ]:
#normalize data to per user numbers
churns_comp_df['churns_page'] = pd.to_numeric(churns_comp_df['churns_page'])

In [ ]:
churns_comp_df.values[0] = churns_comp_df.values[0]/num_churn

In [ ]:
churns_comp_df.values[1] = churns_comp_df.values[1]/num_stay

In [ ]:
#reinsert binary for churning and not (floor division removes the in the churns_page column)
churns_comp_df['churns_page'] = [1,0]

In [ ]:
#Average Stats per user
churns_comp_df

In [ ]:
#DO SOMETHING LIKE THIS
# from https://stackoverflow.com/questions/28576540/how-can-i-normalize-the-data-in-a-range-of-columns-in-my-pandas-dataframe
# cols_to_norm = ['Age','Height']
# survey_data[cols_to_norm] = survey_data[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [ ]:
#Normalize columns
cols_to_norm = ['About','Add Friend', 'Add to Playlist', 'Downgrade', 'Help', 'Home', 'Logout', 
                'NextSong', 'Roll Advert', 'Save Settings', 'Settings', 'Thumbs Down', 'Thumbs Up', 'Upgrade']
churns_comp_df[cols_to_norm] = churns_comp_df[cols_to_norm].apply(lambda x: x / x.sum())


In [ ]:
churns_comp_df

In [ ]:
#adjusting figures with matplotlib
#https://stackoverflow.com/questions/332289/how-do-you-change-the-size-of-figures-drawn-with-matplotlib?answertab=active#tab-top
#the format below allows one change for all plots
plt.rcParams["figure.figsize"] = (15,5)

In [ ]:
non_bin_columns = churns_comp_df[(churns_comp_df != 1) & (churns_comp_df != 0)].loc[0].dropna().index

In [ ]:
churn = plt.bar(x=[x*2-0.5 for x in range(0,len(non_bin_columns))],height=churns_comp_df[non_bin_columns].loc[0], width=0.5, label='churn')
stay = plt.bar(x=[x*2 for x in range(0,len(non_bin_columns))],height=churns_comp_df[non_bin_columns].loc[1], width=0.5, label='stay')
plt.xticks([x*2-0.25 for x in range(0,len(non_bin_columns))],non_bin_columns,rotation=75);
plt.title('Comparison of Sparkify use by Churn/Non-churn users');
plt.legend(handles=[churn,stay])
plt.show()

## Observations
"Cancellation Confirmation" being 1 for the churn row and 0 for the non-churn row affirms that the normalization was done correctly. Quick observations show that those who didn't churn were generally more 'hands on.' They visited the "NextSong" page over 50% more than those who did churn. They visited settings slightly more, the home page, and added almost twice as many friends. Those who didn't churn also hit the "thumbs up" nearly twice as much, likely allowing Sparkify to better match songs for the user.

Unfortunately, users who churned received slightly more ads even with decreased use. This may be part of the problem.

# More specific *churn* definition
Given that we have timestamps available and can easily determine which users downgraded their service, the ML will be better suited with less sparse data on users who "churn." For example, calculating statistics and still only having 1 row per user indicating 'churn' gives us 225 out of 278154 rows to use as positively labeled rows in ML. It might be more useful to have "dowgrades in 48 hours" be our church definition. This allows for more rows, and it would be a step closer to providing a possible intervention window for Sparkify, allowing them to take steps to avoid losing the user from paid service.

In [ ]:
df = df.withColumn('CancelTime',f.when(df['page'] == 'Cancellation Confirmation', df['ts']).otherwise(None))

In [ ]:
df.select('CancelTime').distinct().show()

In [ ]:
df.select('userId','page','churns','ts','datetime','CancelTime').\
        where(df.CancelTime.isNotNull()).\
        show(20)

In [ ]:
all_time = w.partitionBy('userId')\
                .orderBy('ts')\
                .rangeBetween(Window.unboundedPreceding,Window.unboundedFollowing)
# df = df.withColumn('lead',f.lag('CancelTime').over(the_window))

In [ ]:
# define the forward-filled column
filled_column = f.last(df['CancelTime']).over(all_time)

# do the fill
df = df.withColumn('Cancel_Time', filled_column)

In [ ]:
df.select('Cancel_Time','churns','ts','userId','CancelTime').where(df.churns==1).show()

In [ ]:
df = df.drop('CancelTime')

In [ ]:
df = df.withColumn('Until_Cancel',df.Cancel_Time - df.ts)

In [ ]:
df.head()

In [ ]:
df.select('userId','page','Cancel_time','Until_Cancel').where(df.churns==1).show(20)

In [ ]:
df = df.withColumn('CancelDate', f.from_unixtime(df.Cancel_Time/1000))

In [ ]:
check_48 = f.udf(lambda x: 1 if x != None and x <= 172800000 else 0,IntegerType())

In [ ]:
df = df.withColumn('CancelIn48',check_48(df.Until_Cancel))

In [ ]:
df.select('userId').where(df.CancelIn48==1).count()

In [ ]:
df.select('userId','level','datetime','Cancel_Time','CancelDate','Until_Cancel','CancelIn48').where(df.CancelIn48==1).show(50)

In [ ]:
window_48 = w.partitionBy('userId')\
            .orderBy('ts')\
            .rangeBetween(-172800000,0)
window_1_week = w.partitionBy('userId')\
            .orderBy('ts')\
            .rangeBetween(-604800000,0)
window_2_weeks = w.partitionBy('userId')\
            .orderBy('ts')\
            .rangeBetween(-1209600000,0)

In [ ]:
df = df.withColumn('actions_48',f.count(df.page).over(window_48))\
    .withColumn('actions_week', f.count(df.page).over(window_1_week))\
    .withColumn('actions_2weeks', f.count(df.page).over(window_2_weeks))

In [ ]:
advert = f.udf(lambda x: 1 if x =='Roll Advert' else 0, IntegerType())
df = df.withColumn('ad_here',advert(df.page))

In [ ]:
df.select('ad_here','userId','page').show(20)

In [ ]:
df = df.withColumn('ads_2_weeks',f.sum(df.ad_here).over(window_2_weeks))\
    .withColumn('ads_1_week',f.sum(df.ad_here).over(window_1_week))\
    .withColumn('ads_48',f.sum(df.ad_here).over(window_48))

In [ ]:
df.select('ads_2_weeks', 'ads_1_week','ads_48','page','userID','ad_here','Until_Cancel').where(df.ad_here==1).show(80)

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Further possible features
ads per interaction, windows received (e.g. 404 a lot?)

In [ ]:
df.head()

In [ ]:
df.printSchema()

## Explore these features below more

In [ ]:
df.select('itemInSession','CancelIn48').groupBy('CancelIn48').mean().collect()

In [ ]:
#average session item per userId over various time windows
df = df.withColumn('sessionItems_48',f.mean(df.itemInSession).over(window_48))\
        .withColumn('sessionItems_week',f.mean(df.itemInSession).over(window_1_week))\
        .withColumn('sessionItems_2_weeks',f.mean(df.itemInSession).over(window_2_weeks))

In [ ]:
df.head()

In [ ]:
df.select('length', 'CancelIn48').groupBy('CancelIn48').mean().collect()

Length doesn't look like much help

In [ ]:
df.select('status').distinct().count()

In [ ]:
df.select('status', 'CancelIn48').groupBy('status','CancelIn48').count().collect()

200 status = 248544, 6174

307 status = 22569, 615

404 status = 243, 9

#### The statuses all look relatively proportional

# More features to check
- friends added

### Fillna before setting features

In [ ]:
df = df.fillna(-1)
df.head()

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [ ]:
#only maintain features
feat_cols = ['churns','actions_48','actions_week','actions_2weeks'\
             ,'ad_here','ads_2_weeks','ads_1_week','ads_48', 'sessionItems_48'\
             , 'sessionItems_week', 'sessionItems_2_weeks']

In [ ]:
#If retrying in same notebook, uncomment below
# df = df.drop('DescVec')

In [ ]:
assembler = VectorAssembler(inputCols=feat_cols, outputCol='DescVec')
df = assembler.transform(df)

In [ ]:
data = df.select(f.col("CancelIn48").alias("label"), f.col('DescVec').alias("features"))

In [ ]:
train_data, test_data = data.randomSplit([0.75,0.25], seed=42)

# start here?

### Logistic Regression

In [ ]:
#lr = LogisticRegression(family='binomial')
#model_1 = lr.fit(train_data)
#predicts = model_1.transform(test_data)
#predicts.where(predicts.label==1).show()

In [ ]:
# print(predicts.filter((predicts.label == 1) & (predicts.prediction==1)).count())
# print(predicts.filter(predicts.label == 1).count())

### SVC

In [ ]:
# svc = LinearSVC()
# model_svc = svc.fit(train_data)
# svc_pred = model_svc.transform(test_data)
# svc_pred.where(svc_pred.label==1).show()

In [ ]:
# print(svc_pred.filter((svc_pred.label == 1) & (svc_pred.prediction==1)).count())
# print(svc_pred.filter(svc_pred.label == 1).count())

### GBT

In [ ]:
gbt = GBTClassifier()

In [ ]:
# model_gbt = gbt.fit(train_data)
# gbt_pred = model_gbt.transform(test_data)
# gbt_pred.where(gbt_pred.label==1).show()

In [ ]:
print(gbt_pred.filter((gbt_pred.label == 1) & (gbt_pred.prediction==1)).count())
print(gbt_pred.filter(gbt_pred.label == 1).count())

In [ ]:
print(gbt_pred.filter((gbt_pred.label == 0) & (gbt_pred.prediction==0)).count())
print(gbt_pred.filter(gbt_pred.label == 0).count())

In [ ]:
grid = ParamGridBuilder()\
    .addGrid(gbt.maxDepth,[3,5,7])\
    .addGrid(gbt.stepSize,[0.1,0.3])\
    .build()
evaluator = MulticlassClassificationEvaluator(metricName='weightedRecall')

In [ ]:
cv = CrossValidator(estimator=gbt, estimatorParamMaps=grid, evaluator=evaluator)

In [ ]:
cv_model = cv.fit(train_data)

In [ ]:
cv_model.avgMetrics

In [ ]:
results = cv_model.transform(test_data)

In [ ]:
print(results.filter((results.label == 1) & (results.prediction==1)).count())
print(results.filter(results.label == 1).count())

In [ ]:
ding()

In [ ]:
#tuning
#http://spark.apache.org/docs/2.3.2/api/python/pyspark.ml.html#pyspark.ml.tuning.CrossValidator

# Features & Results
### The following tracks features used and number of correctly identified "churn in 48 hours."
#### Original features

##### Logistic Regression
feat_cols = ['churns','actions_48','actions_week','actions_2weeks','ad_here','ads_2_weeks','ads_1_week','ads_48']
**predictions**: 1

#### added 'session_items' avg over time windows
**predictions**: 6

##### SVC

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.